In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
from datetime import date
import matplotlib.pyplot as plt
import seaborn as sns
import requests
pd.options.display.max_rows = 5000 
pd.options.display.max_columns = 200

不同出售情况下的回报预测

想法：如果我想先回收一定程度的本金，比如IPO一开始，回收70%的本金。

In [2]:
def sell_plan(fund, buy_price, w_price, target_price ):
    fund = fund                   #unit = pound
    share_price = buy_price       #unit = pence
    warrant_price = w_price
    
    target_price = target_price
    
    shares = fund*100 / share_price  # 1 pound = 100 pence
    warrants = shares           # if it is 1:1 between shares and warrants
    
#     current_fund = target_price * shares # 当前账户价值
    
    sell_share_to_refund = fund*100 / target_price # 卖出部分股票：这里是100%全部卖出
    
    current_remain_shares = shares - sell_share_to_refund # 当前剩余股票数量
    
    remain_shares_worth = np.round(current_remain_shares*target_price/100) # 剩余股票当前价值
    
    warrant_in_money = [0 if target_price<=warrant_price else target_price-warrant_price][0] # 权证当前是否有价值
    
    remain_warrants_worth = np.round(shares*warrant_in_money/100) # 权证当前价值
    
    total_remain_worth = np.round(shares*warrant_in_money +current_remain_shares*target_price )/100 # 所有剩余价值
    
    total_worth_account = total_remain_worth + fund # 
#  --------------------------------------------------------   
    sell50pct_share_to_refund = fund*0.5*100 / target_price # 卖出部分股票：这里是50%全部卖出
    current_remain_shares_50pct = shares - sell50pct_share_to_refund # 50%后当前剩余股票数量
    remain_shares_worth_50pct = np.round(current_remain_shares_50pct*target_price/100) # 剩余股票当前价值
    total_remain_worth_50pct = np.round(shares*warrant_in_money +current_remain_shares_50pct*target_price )/100 # 所有剩余价值
    
    print('投入的本金: {}镑   当前价格: {}p    账户总价值: {}镑'.format(np.round(fund), target_price,total_worth_account),'\n','\n',
          '*'*50 ,'\n','\n',
          '股票初始购入价格: {}p'.format(share_price),'\n',
          '股票初始购入数量: {}股'.format(np.round(shares)),'\n',
          '权证价格: {}p'.format(warrant_price),'\n',
          '权证初始拥有数量: {}个'.format(np.round(warrants)),'\n','\n',
          '*'*50 ,'\n','\n',
          '当前价格: {}p'.format(target_price),'\n',
          
          '收回全部本金{}镑需出售的股份: {}股'.format(fund, np.round(sell_share_to_refund,0)), '\n','\n',
          '剩余股份数量: {}股'.format(np.round(current_remain_shares)), '\n',
         '剩余股份价值: {}镑'.format(np.round(remain_shares_worth)), '\n',
          '剩余权证数量: {}个'.format(np.round(warrants)), '\n',
         '权证价值: {}镑'.format(remain_warrants_worth), '\n','\n',
         '账户剩余股票和权证总价值: {}镑'.format(total_remain_worth), '\n',
         '*'*50 ,'\n','\n',
          
         '收回50%本金需出售的股份: {}股'.format(np.round(sell50pct_share_to_refund,0)), '\n',
         '50%回本后剩余股份数量: {}股'.format(np.round(current_remain_shares_50pct)), '\n',
         '50%回本剩余股份价值: {}镑'.format(np.round(remain_shares_worth_50pct)), '\n',
         '剩余权证数量: {}个'.format(np.round(warrants)), '\n',
         '权证价值: {}镑'.format(remain_warrants_worth), '\n','\n',
          '50%回本后账户剩余股票和权证总价值: {}镑'.format(total_remain_worth_50pct), '\n',
          
         )

In [8]:
# 回报率计算
x = 5.8
print("当前回报率:{}".format((x-2.4)/2.4))

当前回报率:1.4166666666666667


In [4]:
client_fund = 3000
buy_price = 2.4
w_price = 4.8
current_price = 10
print(sell_plan(client_fund, buy_price, w_price, current_price) )


投入的本金: 3000镑   当前价格: 10p    账户总价值: 19000.0镑 
 
 ************************************************** 
 
 股票初始购入价格: 2.4p 
 股票初始购入数量: 125000.0股 
 权证价格: 4.8p 
 权证初始拥有数量: 125000.0个 
 
 ************************************************** 
 
 当前价格: 10p 
 收回全部本金3000镑需出售的股份: 30000.0股 
 
 剩余股份数量: 95000.0股 
 剩余股份价值: 9500.0镑 
 剩余权证数量: 125000.0个 
 权证价值: 6500.0镑 
 
 账户剩余股票和权证总价值: 16000.0镑 
 ************************************************** 
 
 收回50%本金需出售的股份: 15000.0股 
 50%回本后剩余股份数量: 110000.0股 
 50%回本剩余股份价值: 11000.0镑 
 剩余权证数量: 125000.0个 
 权证价值: 6500.0镑 
 
 50%回本后账户剩余股票和权证总价值: 17500.0镑 

None


In [5]:
def sell_plan_forecast(fund, buy_price, w_price, current_price, recover_pct,target1, target2, target3 ):
    fund = fund                   #初始资金
    share_price = buy_price       #初始买价，单位 pence
    warrant_price = w_price       #权证价格
    recover_pct = recover_pct     #收回本金的百分比
    current_price = current_price #当前价格
    
    shares = fund*100 / share_price  # 初始购买股票数量
    warrants = shares           # 初始拥有权证数量
#  --------------------------------------------------------------------------------   
#     current_fund = current_price * shares # 当前账户价值
    
    sell_share_to_refund = fund*recover_pct*100 / current_price # 卖出股份数量，先卖出回本部分资金：这里是 recover_pct% 卖出比率
    
    current_remain_shares = shares - sell_share_to_refund # 当前剩余股票数量
    
    remain_shares_worth = np.round(current_remain_shares*current_price/100) # 剩余股票当前价值
    
    warrant_in_money = [0 if current_price<=warrant_price else current_price-warrant_price][0] # 权证当前是否有价值
    
    remain_warrants_worth = np.round(shares*warrant_in_money/100) # 权证当前价值
    
    total_remain_worth = np.round(shares*warrant_in_money +current_remain_shares*current_price )/100 # 所有剩余价值
    
    total_worth_account = total_remain_worth + fund*recover_pct # 
# ----------------------------
    post_warrant_in_money = [0 if target1<=warrant_price else target1-warrant_price][0]
    post_sell_remain_worth = np.round(shares*post_warrant_in_money +current_remain_shares*target1 )/100
    
#     -----------------------------

    pre_post_total_worth = post_sell_remain_worth + fund*recover_pct
    
    print('投入的本金: {}镑   当前价格: {}p'.format(np.round(fund), current_price),'\n','\n',
          
          '账户总价值: {}镑    当前投资回报率:{}'.format(total_worth_account, (total_worth_account-fund)/fund),'\n','\n',
          '*'*50 ,'\n','\n',
          
          '股票初始购入价格: {}p'.format(share_price),'\n',
          '股票初始购入数量: {}股'.format(np.round(shares)),'\n',
          '权证价格: {}p'.format(warrant_price),'\n',
          '权证初始拥有数量: {}个'.format(np.round(warrants)),'\n','\n',
          '*'*50 ,'\n','\n',
          '当前价格: {}p'.format(current_price),'\n',
          
          '收回资金{}镑需出售的股份: {}股'.format(fund*recover_pct, np.round(sell_share_to_refund,0)), '\n','\n',
          '剩余股份数量: {}股'.format(np.round(current_remain_shares)), '\n',
         '剩余股份价值: {}镑'.format(np.round(remain_shares_worth)), '\n','\n',
          '剩余权证数量: {}个'.format(np.round(warrants)), '\n',
         '权证价值: {}镑'.format(remain_warrants_worth), '\n','\n',
         '账户剩余股票和权证总价值: {}镑'.format(total_remain_worth), '\n',
         '*'*50 ,'\n','\n',
          '*'*50 ,'\n','\n',
         '假设第一次卖出部分回本{}镑之后, 股价上升到{}p'.format(fund*recover_pct, target1),'\n',
         '剩余股份数量: {}股'.format(np.round(current_remain_shares)), '\n',
         '剩余权证数量: {}个'.format(np.round(warrants)), '\n','\n',
         '卖过一次后，剩余股票权证在target下的价值：{}镑'.format(post_sell_remain_worth),'\n','\n',
         '加上前次的卖出回款，两次操作总收益：{}镑'.format(pre_post_total_worth),'\n','\n',
         '两次操作的总投资回报率：{}'.format((pre_post_total_worth-fund)/fund))

In [6]:
client_fund = 1000
buy_price = 2.4
w_price = 4.8
# -------------------------------------
current_price = 5.8   # 当前价格
recover_amount = 0 # 回收本金，单位：镑
recover_pct = (recover_amount / client_fund)
target1 = 10         # 未来价格
sell_plan_forecast(client_fund, buy_price, w_price, current_price, recover_pct, target1,0,0)

投入的本金: 1000镑   当前价格: 5.8p 
 
 账户总价值: 2833.33镑    当前投资回报率:1.83333 
 
 ************************************************** 
 
 股票初始购入价格: 2.4p 
 股票初始购入数量: 41667.0股 
 权证价格: 4.8p 
 权证初始拥有数量: 41667.0个 
 
 ************************************************** 
 
 当前价格: 5.8p 
 收回资金0.0镑需出售的股份: 0.0股 
 
 剩余股份数量: 41667.0股 
 剩余股份价值: 2417.0镑 
 
 剩余权证数量: 41667.0个 
 权证价值: 417.0镑 
 
 账户剩余股票和权证总价值: 2833.33镑 
 ************************************************** 
 
 ************************************************** 
 
 假设第一次卖出部分回本0.0镑之后, 股价上升到10p 
 剩余股份数量: 41667.0股 
 剩余权证数量: 41667.0个 
 
 卖过一次后，剩余股票权证在target下的价值：6333.33镑 
 
 加上前次的卖出回款，两次操作总收益：6333.33镑 
 
 两次操作的总投资回报率：5.33333
